In [1]:
import duckdb
import pandas as pd
from IPython.display import display
import plotly.express as px
from eda_support_functions import *


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [3]:
wikipedia_movie_financial_data = pd.read_csv('../data/retrieved_data/wikipedia/movie_financial_data.csv').drop(columns='Unnamed: 0')
wikipedia_movie_financial_data_usd = pd.read_csv('../data/retrieved_data/wikipedia/movie_financial_data_usd.csv')


movie = pd.read_csv('../data/retrieved_data/tmdb/movies.csv')
mojo = pd.read_csv('../data/manual_data/parsed_mojo.csv').rename(
    columns={'Unnamed: 0': 'imdb_id', 'worldwide': 'revenue_mojo'})

In [4]:
mojo

,imdb_id,international,revenue_mojo,domestic,error
0,tt0421045,0.0,0.0,0.0,NaN
1,tt1091207,0.0,0.0,0.0,NaN
2,tt0473024,0.0,7009668.0,7009668.0,NaN
3,tt1879066,0.0,0.0,0.0,NaN
4,tt0119053,0.0,6136856.0,6136856.0,NaN
...,...,...,...,...,...
11167,tt1401668,NaN,NaN,NaN,RetryError[<Future at 0x3e6ae1f49040 state=fin...
11168,tt1972571,NaN,NaN,NaN,RetryError[<Future at 0x3e6ad11c2d30 state=fin...
11169,tt0114959,NaN,NaN,NaN,RetryError[<Future at 0x3e5a6f760580 state=fin...
11170,tt0084290,NaN,NaN,NaN,RetryError[<Future at 0x3e29315cd310 state=fin...


In [5]:
manual_adjustments = pd.read_csv('../data/manual_data/manual_adjustments.csv')
manual_adjustments = manual_adjustments[["movie_id", "revenue_fix","budget_fix"]].fillna(0)

In [6]:
# Print how many movies in wikipedia_movie_financial_data
print(f'Number of movies in wikipedia_movie_financial_data: {len(wikipedia_movie_financial_data)}')
unique_imdb_ids = wikipedia_movie_financial_data['tmdb_id'].unique()
print(f'Number of unique imdb_ids in wikipedia_movie_financial_data: {len(unique_imdb_ids)}')


Number of movies in wikipedia_movie_financial_data: 43355
Number of unique imdb_ids in wikipedia_movie_financial_data: 43355


In [7]:
# #load json from url into dataframe
# import requests
# import json
# url = 'https://ffbe7889f116f85ae79fd90903cf7cccfd41509ca2fe525c2cbda26-apidata.googleusercontent.com/download/storage/v1/b/movie-dataset-291206-movie-data-results/o/20240714_094035_aggregated_results.json?jk=AYBlUPBJApueiHReA-hfruhHxgh8ghoMyS56g-SXJFQSLSg4fymkM4dgnxV6XZyyLQuP54YFFyIEjVmx_Q4oG5jGu3cxPMi2UMXB2FR_DARj-6blknNhuXaBHp6tCXf0HYtxwrC7yklq7U2a8NBQFFf9sLik7i5I1JNWLEe7tDOrU1rAKTNmstj9T9EAWAkX8ULAKB51HFE_mf1TN6NsCEnt_wvWpnY7kufRoI2c-NiLrckk5RGoUnXIiwQF5PP1BIRyIMTHrPm_qeE9s8hnSViNFc0rfhHO0BsIe559FqnDEdIhux07QUfh2DARX8dXKo0Mmb_YHCx368SaexGNydOzAGkfPwhLV9T914KDMwOOOp-H2BPm1kKFKyUc9zdzVQx-dA4RSzvvZn7Ti6n5xPpDhHfgMQe-zI0aRezkwpS6UvdCilkarNR5Mz-5M9Pst9Mqil25h-_2D88ALyROA5090E-iyursRs8smId5NE21LkTHcIBZb1-L9XMhsaJw701dzR7AaKZO2vdh0IouEiBJyWCFTmjDiZ2QJXlM6ssEWyMEQZGl5f17BiXTI2RzO-XnuaWPgJ3uulaOa7OijKsMQYt9mZg6G5usx7B8oDwV2TtkTepqBJvMztVC1iVJ2HgE3VR-tPNkP3TXvsELbYcX4OM1XlOJ-ombTOamS-nQsbH7NKLUCxF3yhOGtFbtMaHt7A9Uv6jr0XKkXiw3YrW9H-AdVMrzqbHKzU5PrWJV0NC_JVkI6fK8t_XweMN_Du--nZOTtdsYLkeiR3iZ2LeBxneKkxgbnAwKBnv0gVRtcrP54C55EwKWkdrf7yAnl6DQBBRZRCZCcZwliv01gHXG4AcKbYLCl3dPotdBR_VvGMEkq6MZ1GQI639GSYhl_38fUBix884nosRXiog4V5m8c__hz1boaESxkpo89QO8Y-IFgFYm1_uMfzxxwHAn0hkRjD692jfEowJcn_2p7i3lN-6CnM6aZONYoYCfTVMIfFm8IuZKWyCHYABunDXgDd5iquPYOrLFFFOFPcqd9RF-jQJQ_P49NoA_ITCEGQK44veox7bbpI0Pbju6zHpTswIFEESbyLbHFmFjLiVMQGVCP51J5Do79r-aT7DkKNTskdrm4mzSpLN9aPEN5CxgqB_cNj9xaxB8W36AIximjr08S9_6un6P4dxUFFmconT0bWxel1kcQZV-zyvt6mSGuhegE-fXPFY8bVaI4dYtQYURdhf2RVC8&isca=1'
# response = requests.get(url)
# data = response.json()
# # print(data)
# mojo = pd.read_json(json.dumps(data), orient='index')

# mojo.to_csv('../data/manual_data/parsed_mojo.csv')

# Wikipedia

In [8]:
# Check wikipedia_movie_financial_data for duplicates on tmdb_id
display(wikipedia_movie_financial_data[wikipedia_movie_financial_data.duplicated(subset=['tmdb_id'], keep=False)])

# How many unique tmdb_ids are duplicated?
print(wikipedia_movie_financial_data[wikipedia_movie_financial_data.duplicated(subset=['tmdb_id'], keep=False)]['tmdb_id'].nunique())

,url,tmdb_id,budget_original_string,budget_currency,budget_min,budget_max,budget_avg,budget_error,box_office_original_string,box_office_currency,box office_min,box_office_max,box_office_avg,box_office_error,external_links,error,box_office_min


0


In [9]:
wikipedia_movie_financial_data.groupby(by=['budget_currency']).size().sort_values(ascending=False).head(20)

budget_currency
USD    10518
INR      828
EUR      601
GBP      447
AUD      217
CAD      110
BRL      106
JPY       79
SEK       54
KRW       44
RUB       43
HKD       39
CZK       31
DEM       28
ESP       27
PHP       24
NOK       19
NGN       16
DKK       14
CHF       14
dtype: int64

In [10]:
wikipedia_movie_financial_data.groupby(by=['box_office_currency']).size().sort_values(ascending=False).head(20)

box_office_currency
USD    13193
INR      919
HKD      290
JPY      254
AUD      176
GBP      167
EUR      123
ITL      115
BRL       86
PHP       57
SEK       32
RUB       31
ESP       28
NGN       24
CHF       23
CZK       18
CAD       17
DEM       10
TRY       10
KRW        9
dtype: int64

In [11]:
wikipedia_movie_financial_data[(wikipedia_movie_financial_data['budget_currency'].isnull()) & (wikipedia_movie_financial_data['budget_original_string'].notnull())]['budget_original_string'].value_counts()[:70]

budget_original_string
Unknown                                                               8
IL3 million                                                           7
P208,120,000                                                          2
MYR 2.5 million                                                       2
ƒ 5 million                                                           1
MYR 3 million                                                         1
-                                                                     1
Rs 1,50,000                                                           1
Undisclosed                                                           1
Kadr Film                                                             1
7.6 million [1]                                                       1
3 million[1]                                                          1
12 million FF                                                         1
Rp. 80 million                           

In [12]:
print(f"Number of revenue_mojo columns not with 0 values: {(mojo['revenue_mojo'] != 0).sum()}")
print(f"Number of cases with error: {(~mojo['error'].isna()).sum()}")

Number of revenue_mojo columns not with 0 values: 5931
Number of cases with error: 23


In [13]:
mojo = mojo[['imdb_id', 'revenue_mojo', 'error']]
movie = movie.merge(mojo, on='imdb_id', how='left')
movie['revenue_mojo'] = movie['revenue_mojo'].fillna(0)

wikipedia_movie_financial_data_usd_raw = wikipedia_movie_financial_data_usd.copy()
wikipedia_movie_financial_data_usd = wikipedia_movie_financial_data_usd[['movie_id', 'budget_avg_usd', 'box_office_avg_usd', 'url']].rename(columns={'budget_avg_usd': 'budget_wiki', 'box_office_avg_usd': 'revenue_wiki','url' : 'url_wiki'})
movie = movie.merge(wikipedia_movie_financial_data_usd, on='movie_id', how='left')
movie = movie.merge(manual_adjustments, on='movie_id', how='left')
movie['budget_wiki'] = movie['budget_wiki'].fillna(0).astype(int)
movie['revenue_wiki'] = movie['revenue_wiki'].fillna(0).astype(int)
movie['revenue_fix'] = movie['revenue_fix'].fillna(0).astype(int)
movie['budget_fix'] = movie['budget_fix'].fillna(0).astype(int)




In [14]:
# Replace the 0 values with the values from the wikipedia dataset
movie['budget_synthetic'] = movie['budget_fix'].replace(0, np.nan)
movie['budget_synthetic'] = movie['budget_synthetic'].fillna(movie['budget']).replace(0, np.nan)
movie['budget_synthetic'] = movie['budget_synthetic'].fillna(movie['budget_wiki']).fillna(0).astype(int)

# Create synthetic box office
movie['revenue_synthetic'] = movie['revenue_fix'].replace(0, np.nan)
movie['revenue_synthetic'] = movie['revenue_synthetic'].fillna(movie['revenue_mojo']).replace(0, np.nan)
movie['revenue_synthetic'] = movie['revenue_synthetic'].fillna(movie['revenue']).replace(0, np.nan)
movie['revenue_synthetic'] = movie['revenue_synthetic'].fillna(movie['revenue_wiki']).fillna(0).astype(int)

In [15]:
print("TMDB Data")
print(f"Number of tmdb revenue column not 0: {(movie['revenue'] != 0).sum()}")
print(f"Number of tmdb budget column not 0: {(movie['budget'] != 0).sum()}")
print(f"Number of revenue and budget not 0 : {((movie['revenue'] != 0) & (movie['budget'] != 0)).sum()}")

print("Mojo Enriched")
print(f"Number of mojo revenue column not 0: {(movie['revenue_mojo'] != 0).sum()}")
print(f"Number of revenue not 0: {((movie['revenue'] != 0) | (movie['revenue_mojo'] != 0)).sum()}")
print(f"Number of revenue and budget not 0 : {(((movie['revenue'] != 0) | (movie['revenue_mojo'] != 0)) & (movie['budget'] != 0)).sum()}")

print("Wikipedia Enriched")
print(f"Number of revenue column not 0: {(movie['revenue_synthetic'] != 0).sum()}")
print(f"Number of budget column not 0: {(movie['budget_synthetic'] != 0).sum()}")
print(f"Number of revenue and budget not 0 : {((movie['revenue_synthetic'] != 0) & (movie['budget_synthetic'] != 0)).sum()}")


TMDB Data
Number of tmdb revenue column not 0: 11549
Number of tmdb budget column not 0: 11719
Number of revenue and budget not 0 : 7654
Mojo Enriched
Number of mojo revenue column not 0: 5907
Number of revenue not 0: 14633
Number of revenue and budget not 0 : 8006
Wikipedia Enriched
Number of revenue column not 0: 19806
Number of budget column not 0: 16563
Number of revenue and budget not 0 : 12190


In [16]:
compare = movie[(movie['revenue'] != 0) & (movie['revenue_mojo'] != 0)].copy()
compare['revenue_diff'] = compare['revenue'] - compare['revenue_mojo']

# Calulate metrics
compare['revenue_diff'] = compare['revenue'] - compare['revenue_mojo']
compare['revenue_ratio'] = compare['revenue'] / compare['revenue_mojo']
compare['revenue_diff_ratio'] = compare['revenue_diff'] / compare['revenue_mojo']

# Display
display(compare[['imdb_id', 'movie_id', 'revenue', 'revenue_mojo', 'revenue_diff', 'revenue_ratio', 'revenue_diff_ratio']].sort_values(by='revenue_diff_ratio', ascending=False).head(10))

# Describe
compare[['revenue_diff_ratio','revenue_ratio', 'revenue_diff']].describe(percentiles=[0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.97, 0.98, 0.99]).style.format("{:.2f}")

,imdb_id,movie_id,revenue,revenue_mojo,revenue_diff,revenue_ratio,revenue_diff_ratio
63774,tt0080516,13550,12000000,147.0,11999853.0,81632.653061,81631.653061
47307,tt0066769,10514,12376563,430.0,12376133.0,28782.704651,28781.704651
57113,tt0070723,12101,3600000,210.0,3599790.0,17142.857143,17141.857143
64944,tt0080310,13783,8200000,948.0,8199052.0,8649.789030,8648.789030
49653,tt0074695,10839,1509000,201.0,1508799.0,7507.462687,7506.462687
29096,tt0068638,5916,36734619,9588.0,36725031.0,3831.311952,3830.311952
7456,tt0082464,16124,1400000,394.0,1399606.0,3553.299492,3552.299492
59518,tt5125414,496681,7082000,3403.0,7078597.0,2081.104907,2080.104907
6665,tt0080117,233,23000000,14492.0,22985508.0,1587.082528,1586.082528
10713,tt0071222,16938,4000000,2686.0,3997314.0,1489.203276,1488.203276


,revenue_diff_ratio,revenue_ratio,revenue_diff
count,2823.00,2823.00,2823.00
mean,57.78,58.78,92478.87
std,1678.26,1678.26,9821291.62
min,-1.00,0.00,-231142888.00
1%,-1.00,0.00,-21066273.00
5%,-0.65,0.35,-3263938.60
10%,-0.26,0.74,-1043686.40
25%,-0.01,0.99,-35342.50
50%,0.00,1.00,0.00
75%,0.00,1.00,0.00


In [17]:
null_percentage_all = movie.isnull().mean() * 100

zero_percentage_all = (movie == 0).mean() * 100

null_zero_percentage_all = pd.DataFrame({'Null Percentage': null_percentage_all, 'Zero Percentage': zero_percentage_all})
print(null_zero_percentage_all)

                   Null Percentage  Zero Percentage
movie_id                  0.000000         0.000000
title                     0.000000         0.000000
original_title            0.000000         0.000000
original_language         0.000000         0.000000
overview                  5.753660         0.000000
tagline                  64.028894         0.000000
release_date              0.239798         0.000000
runtime                   0.000000         5.539026
budget                    0.000000        82.653167
revenue                   0.000000        82.904806
poster_path               3.554036         0.000000
imdb_id                   0.455911         0.000000
facebook_id              95.750255         0.000000
instagram_id             98.290333         0.000000
twitter_id               97.939518         0.000000
wikidata_id              88.353538         0.000000
revenue_mojo              0.000000        91.256272
error                    99.965955         0.000000
budget_wiki 

In [18]:
movie = movie[(movie['revenue_synthetic'] != 0) & (movie['budget_synthetic'] != 0)]

In [19]:
movie[['budget', 'budget_wiki', 'budget_synthetic', 'revenue', 'revenue_mojo', 'revenue_wiki','revenue_synthetic']].describe().drop('count').style.format({
    'budget': "${:,.0f}",
    'budget_wiki': "${:,.0f}",
    'budget_synthetic': "${:,.0f}",
    'revenue': "${:,.0f}",
    'revenue_mojo': "${:,.0f}",
    'revenue_wiki': "${:,.0f}",
    'revenue_synthetic': "${:,.0f}"
})

,budget,budget_wiki,budget_synthetic,revenue,revenue_mojo,revenue_wiki,revenue_synthetic
mean,"$15,751,448","$16,483,502","$17,874,772","$41,679,766","$11,745,903","$45,084,540","$43,865,711"
std,"$29,254,324","$30,875,857","$29,479,520","$118,894,140","$64,943,730","$122,418,646","$119,020,574"
min,$0,$0,$1,$0,$0,$0,$1
25%,$0,"$1,711,542","$3,000,000",$0,$0,"$1,013,515","$1,373,416"
50%,"$5,000,000","$7,000,000","$8,000,000","$3,796,807",$0,"$7,800,000","$7,834,857"
75%,"$19,000,000","$19,652,214","$20,000,000","$30,721,501","$7,192","$35,800,000","$33,172,674"
max,"$379,000,000","$1,107,215,353","$500,000,000","$2,923,706,026","$2,923,706,026","$2,923,000,000","$2,923,706,026"


In [20]:
movie['synthetic_ratio'] = movie['revenue_synthetic'] / movie['budget_synthetic']

In [21]:
movie[['movie_id','title','url_wiki','original_language','revenue', 'revenue_wiki', 'revenue_mojo', 'revenue_synthetic','budget', 'budget_wiki', 'budget_synthetic','synthetic_ratio']].sort_values(by='synthetic_ratio', ascending=False).to_clipboard()

In [22]:
movie[['budget', 'budget_wiki','budget_synthetic']].corr().style.format("{:.2f}").background_gradient(cmap='coolwarm')

,budget,budget_wiki,budget_synthetic
budget,1.00,0.78,0.95
budget_wiki,0.78,1.00,0.83
budget_synthetic,0.95,0.83,1.00


In [23]:
movie[['revenue', 'revenue_mojo', 'revenue_wiki','revenue_synthetic']].corr().style.format("{:.2f}").background_gradient(cmap='coolwarm')

,revenue,revenue_mojo,revenue_wiki,revenue_synthetic
revenue,1.00,0.49,0.96,0.99
revenue_mojo,0.49,1.00,0.48,0.50
revenue_wiki,0.96,0.48,1.00,0.96
revenue_synthetic,0.99,0.50,0.96,1.00


In [24]:
movie[['runtime', 'budget', 'revenue']].describe().drop('count').style.format({
    'runtime': "{:.0f}",
    'budget': "${:,.0f}",
    'revenue': "${:,.0f}"
})

,runtime,budget,revenue
mean,109,"$15,751,448","$41,679,766"
std,21,"$29,254,324","$118,894,140"
min,0,$0,$0
25%,94,$0,$0
50%,104,"$5,000,000","$3,796,807"
75%,118,"$19,000,000","$30,721,501"
max,360,"$379,000,000","$2,923,706,026"
